# Counterfactuals benchmark on tabular datasets

In [1]:
from tensorflow.compat.v1 import enable_eager_execution
from tensorflow import executing_eagerly
import os
enable_eager_execution()

BASE_PATH = "./counterfactuals"
print("Current working directory:", os.getcwd())

2025-07-12 12:57:21.503200: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-12 12:57:22.023561: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-12 12:57:22.023615: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-12 12:57:22.026238: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-12 12:57:22.248362: I tensorflow/core/platform/cpu_feature_g

Current working directory: /home/ahmed/prototype


## Imports and preprocessing

In [2]:
# Install the dev version of the Alibi package if not already installed
try:
    from alibi import __version__ as alibi_version
    print(f"Alibi version: {alibi_version}")
except ImportError:
    print("Alibi package not found, installing...")
    # Install the dev version of Alibi
    !pip install git+https://github.com/SeldonIO/alibi.git > /dev/null


import logging

alibi_logger = logging.getLogger("alibi")
alibi_logger.setLevel("CRITICAL")


print(f"Is TensorFlow running in eager execution mode? -----→ {executing_eagerly()}")
!nvidia-smi -L

/home/ahmed/prototype/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Alibi version: 0.9.7.dev0
Is TensorFlow running in eager execution mode? -----→ True
GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU (UUID: GPU-ed7340f2-1910-df12-4a83-29feeba52695)


In [3]:
from datetime import datetime

if not os.path.exists(BASE_PATH):
    os.makedirs(BASE_PATH)


date = datetime.now().strftime('%Y-%m-%d')
EXPERIMENT_PATH = f"{BASE_PATH}/diabetes_{date}"
MODELS_EXPERIMENT_PATH = f"{BASE_PATH}/diabetes_2020-09-09"
if not os.path.exists(EXPERIMENT_PATH):
    os.makedirs(EXPERIMENT_PATH)
    

## Data import and preprocessing

In [4]:
import json
# import pickle
# import time
# from matplotlib import offsetbox
# from matplotlib.colors import ListedColormap
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.layers import Dense, Add, Input, ActivityRegularization, Concatenate, Multiply
from tensorflow.keras import optimizers, Model, regularizers, Input
from imblearn.over_sampling import SMOTE

from tensorflow.keras.models import Sequential
from tensorflow.random import set_seed
# from tensorflow.keras.models import load_model
import os

print("Current working directory:", os.getcwd())

INITIAL_CLASS = 0
DESIRED_CLASS = 1
N_CLASSES = 2
n_training_iterations = 10


np.set_printoptions(precision=2)
set_seed(2020)
np.random.seed(2020)

# German Credit dataset

def preprocess_data_german(df, target_column="Outcome"):
    """
    Preprocess the German Credit dataset by encoding categorical variables and splitting the data into 
    train, test, and user simulation sets.
    
    Returns a dictionary with processed train, test, and user datasets.
    """
    
    # Assign meaningful column names
    df.columns = [
        'Status', 'Month', 'Credit_History', 'Purpose', 'Credit_Amount',
        'Savings', 'Employment', 'Installment_Rate', 'Personal_Status', 'Other_Debtors',
        'Residence_Duration', 'Property', 'Age', 'Other_Installment_Plans', 'Housing',
        'Existing_Credits', 'Job', 'Num_Liable_People', 'Telephone', 'Foreign_Worker',
        'Outcome'
    ]
    
    # Mapping categorical features to more meaningful values
    status_mapping = { 'A11': '< 0 DM', 'A12': '0 <= ... < 200 DM', 'A13': '>= 200 DM / salary assignments for at least 1 year', 'A14': 'no checking account' }
    credit_history_mapping = { 'A30': 'no credits taken/ all credits paid back duly', 'A31': 'all credits at this bank paid back duly', 'A32': 'existing credits paid back duly till now', 'A33': 'delay in paying off in the past', 'A34': 'critical account/other credits existing' }
    savings_mapping = { 'A61': '< 100 DM', 'A62': '100 <= ... < 500 DM', 'A63': '500 <= ... < 1000 DM', 'A64': '>= 1000 DM', 'A65': 'unknown/no savings account' }
    employment_mapping = { 'A71': 'unemployed', 'A72': '< 1 year', 'A73': '1 <= ... < 4 years', 'A74': '4 <= ... < 7 years', 'A75': '>= 7 years' }
    personal_status_mapping = { 'A91': 'male: divorced/separated', 'A92': 'female: divorced/separated/married', 'A93': 'male: single', 'A94': 'male: married/widowed', 'A95': 'female: single' }
    other_debtors_mapping = { 'A101': 'none', 'A102': 'co-applicant', 'A103': 'guarantor' }
    property_mapping = { 'A121': 'real estate', 'A122': 'building society savings agreement/life insurance', 'A123': 'car or other, not in attribute 6', 'A124': 'unknown/no property' }
    other_installment_plans_mapping = { 'A141': 'bank', 'A142': 'stores', 'A143': 'none' }
    housing_mapping = { 'A151': 'rent', 'A152': 'own', 'A153': 'for free' }
    telephone_mapping = { 'A191': 'none', 'A192': 'yes, registered under the customer\'s name' }
    foreign_worker_mapping = { 'A201': 'yes', 'A202': 'no' }

    # Apply mappings
    df['Status'] = df['Status'].map(status_mapping)
    df['Credit_History'] = df['Credit_History'].map(credit_history_mapping)
    df['Savings'] = df['Savings'].map(savings_mapping)
    df['Employment'] = df['Employment'].map(employment_mapping)
    df['Personal_Status'] = df['Personal_Status'].map(personal_status_mapping)
    df['Other_Debtors'] = df['Other_Debtors'].map(other_debtors_mapping)
    df['Property'] = df['Property'].map(property_mapping)
    df['Other_Installment_Plans'] = df['Other_Installment_Plans'].map(other_installment_plans_mapping)
    df['Housing'] = df['Housing'].map(housing_mapping)
    df['Telephone'] = df['Telephone'].map(telephone_mapping)
    df['Foreign_Worker'] = df['Foreign_Worker'].map(foreign_worker_mapping)

    # Encode ordinal columns
    ordinal_cols = ['Status', 'Credit_History', 'Savings', 'Employment']
    le = LabelEncoder()
    for col in ordinal_cols:
        df[col] = le.fit_transform(df[col])

    # One-hot encode nominal columns
    nominal_columns = ['Purpose', 'Personal_Status', 'Other_Debtors', 'Property', 
                       'Other_Installment_Plans', 'Housing', 'Job', 'Telephone', 'Foreign_Worker']
    df = pd.get_dummies(df, columns=nominal_columns, drop_first=True)

    # Process target variable
    Y = df[target_column].replace(1, 0).replace(2, 1)
    X = df.drop(columns=[target_column])

    # Get final feature set
    # list all features
    immutable_features = set(X.columns) - set(['Status', 'Credit_History'])
    

    mutable_features = set(X.columns) - set(immutable_features)
    mutable_features = list(mutable_features)

    features = list(mutable_features) + list(immutable_features)

    return  X, Y, features, immutable_features, mutable_features
    
# =========================================================


# Make sure 'german.csv' is in your project directory
df = pd.read_csv('statlog_german_credit_data/german.data', sep=' ', skiprows=1, header=None)
x,y, features, immutable_features, mutable_features = preprocess_data_german(df)

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=2020)

standard_scaler = StandardScaler()
X_train = standard_scaler.fit_transform(X_train)
X_test = standard_scaler.transform(X_test)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)



Current working directory: /home/ahmed/prototype


In [5]:
def compute_reconstruction_error(x, autoencoder):
    """Compute the reconstruction error for a given autoencoder and data points."""
    preds = autoencoder.predict(x)
    preds_flat = preds.reshape((preds.shape[0], -1))
    x_flat = x.reshape((x.shape[0], -1))
    return np.linalg.norm(x_flat - preds_flat, axis=1)

def format_metric(metric):
    """Return a formatted version of a metric, with the confidence interval."""
    return f"{metric.mean():.3f} ± {1.96*metric.std()/np.sqrt(len(metric)):.3f}"

def compute_metrics(samples, counterfactuals, latencies, classifier, autoencoder,
                    batch_latency=None):
    """ Summarize the relevant metrics in a dictionary. """
    reconstruction_error = compute_reconstruction_error(counterfactuals, autoencoder)
    delta = np.abs(samples-counterfactuals)
    l1_distances = delta.reshape(delta.shape[0], -1).sum(axis=1)
    prediction_gain = (
        classifier.predict(counterfactuals)[:, DESIRED_CLASS] - 
        classifier.predict(samples)[:, DESIRED_CLASS]
    )

    metrics = dict()
    metrics["reconstruction_error"] = format_metric(reconstruction_error)
    metrics["prediction_gain"] = format_metric(prediction_gain)
    metrics["sparsity"] = format_metric(l1_distances)
    metrics["latency"] = format_metric(latencies)
    batch_latency = batch_latency if batch_latency else sum(latencies)
    metrics["latency_batch"] = f"{batch_latency:.3f}"

    return metrics

def save_experiment(method_name, samples, counterfactuals, latencies, 
                    batch_latency=None):
    """Create an experiment folder and save counterfactuals, latencies and metrics."""
    if not os.path.exists(f"{EXPERIMENT_PATH}/{method_name}"):
        os.makedirs(f"{EXPERIMENT_PATH}/{method_name}")   

    np.save(f"{EXPERIMENT_PATH}/{method_name}/counterfactuals.npy", counterfactuals)
    np.save(f"{EXPERIMENT_PATH}/{method_name}/latencies.npy", latencies)

    metrics = compute_metrics(samples, counterfactuals, latencies, classifier, autoencoder)
    json.dump(metrics, open(f"{EXPERIMENT_PATH}/{method_name}/metrics.json", "w"))
    pprint(metrics)

Train classifier

In [6]:
set_seed(2020)
np.random.seed(2020)

def create_classifier(input_shape):
    """Define and compile a neural network binary classifier.""" 
    model = Sequential([
        Dense(20, activation='relu', input_shape=input_shape),
        Dense(20, activation='relu'),
        Dense(2, activation='softmax'),
    ], name="classifier")
    optimizer = optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(optimizer, 'binary_crossentropy', ['accuracy'])
    return model

classifier = create_classifier((x.shape[1],))

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

sm = SMOTE(random_state=2020)
X_res, y_res = sm.fit_resample(X_train, y_train)

y_res = to_categorical(y_res)
# Train the classifier
training = classifier.fit(X_res, y_res, batch_size=32, epochs=200, verbose=0,
                          validation_data=(X_test, y_test),)
print(f"Training: loss={training.history['loss'][-1]:.4f}, "
      f"accuracy={training.history['accuracy'][-1]:.4f}")
print(f"Validation: loss={training.history['val_loss'][-1]:.4f}, "
      f"accuracy={training.history['val_accuracy'][-1]:.4f}")

classifier.save(f"{EXPERIMENT_PATH}/classifier.keras")



2025-07-12 12:57:35.437062: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-12 12:57:35.728999: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-12 12:57:35.729100: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-12 12:57:35.732424: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-12 12:57:35.732557: I tensorflow/compile

Training: loss=0.2331, accuracy=0.9213
Validation: loss=0.7291, accuracy=0.6650


In [7]:
from sklearn.metrics import classification_report
import numpy as np

# Convert probabilities to class labels
y_pred = classifier.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

print(classification_report(y_test_classes, y_pred_classes))


7/7 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

           0       0.78      0.72      0.75       140
           1       0.45      0.53      0.49        60

    accuracy                           0.67       200
   macro avg       0.62      0.63      0.62       200
weighted avg       0.68      0.67      0.67       200



## Estimate density with the reconstruction error of a (denoising) autoencoder


In [8]:
def add_noise(x, noise_factor=1e-6):
    x_noisy = x + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x.shape) 
    return x_noisy

    
def create_autoencoder(in_shape=(x.shape[1],)):
    input_ = Input(shape=in_shape) 

    x = Dense(32, activation="relu")(input_)
    encoded = Dense(8)(x)
    x = Dense(32, activation="relu")(encoded)
    decoded = Dense(in_shape[0], activation="tanh")(x)

    autoencoder = Model(input_, decoded)
    optimizer = optimizers.Nadam()
    autoencoder.compile(optimizer, 'mse')
    return autoencoder

autoencoder = create_autoencoder()
training = autoencoder.fit(
    add_noise(X_train), X_train, epochs=100, batch_size=32, shuffle=True, 
    validation_data=(X_test, X_test), verbose=0
)
print(f"Training loss: {training.history['loss'][-1]:.4f}")
print(f"Validation loss: {training.history['val_loss'][-1]:.4f}")

n_samples = 1000
# Compute the reconstruction error of noise data
samples = np.random.randn(n_samples, X_train.shape[1])
reconstruction_error_noise = compute_reconstruction_error(samples, autoencoder)

# Save and print the autoencoder metrics
reconstruction_error = compute_reconstruction_error(X_test, autoencoder)
autoencoder_metrics = {
    "reconstruction_error": format_metric(reconstruction_error),
    "reconstruction_error_noise": format_metric(reconstruction_error_noise),
}
json.dump(autoencoder_metrics, open(f"{EXPERIMENT_PATH}/autoencoder_metrics.json", "w"))
pprint(autoencoder_metrics)

autoencoder.save(f"{EXPERIMENT_PATH}/autoencoder.keras")


Training loss: 0.5158
Validation loss: 0.6319
7/7 [==============================] - 0s 3ms/step
{'reconstruction_error': '4.476 ± 0.253',
 'reconstruction_error_noise': '5.846 ± 0.048'}
